In [ ]:
import os
import time

import cv2

import numpy as np
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

In [ ]:
def get_object_detection_model(num_classes):

    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")

    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    return model

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
model = get_object_detection_model(14)

In [ ]:
model.load_state_dict(torch.load('training/faster_rcnn_weights_best.pt'))
model.to(device)

In [ ]:
total_time = 0
for directory, _, files in os.walk('../../dataset/train/images'):
    for i in range(100):
        image = cv2.imread(f'{directory}/{files[i]}', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        image = torch.tensor(image)
        image = image.permute(2, 0, 1)

        model.eval()
        with torch.no_grad():
            image_time = time.time()
            prediction = model([image.to(device)])[0]
            total_time += (time.time() - image_time)

print(total_time / 100)